<a href="https://colab.research.google.com/github/uadyf/Bootcamp-2021/blob/main/tugas_akhir/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get File
(upload to google colab)

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
def con(path):
  conn = sqlite3.connect(str(path))
  return conn

def replaceNone(list_):
  replaced = []
  for i in range(len(list_)):
    new = [np.nan if val==None else val for val in list_[i]]
    replaced.append(new)
  return replaced

def query(q, path):
  df = pd.read_sql_query(str(q), con(str(path)))
  col = df.columns

  replace_nan = replaceNone(df.values)
  df_new = pd.DataFrame(replace_nan, columns=col)
  return df_new


In [ ]:
q_user = '''
        SELECT * FROM user
        '''
q_tweet = '''
          SELECT * FROM tweet
          '''
file_ = '/content/Basisdata(raw).db'

df_user = query(q_user, file_)
df_tweet = query(q_tweet, file_)

## preprocess tweet

In [2]:
import re
import string

In [ ]:
dT = df_tweet
dT

,tweetid,createddate,tweet
0,1357479018472239104,Thu Feb 04 23:59:53 +0000 2021,"RT @mhmdadipati: @farisandani Dari orang"" yang..."
1,1357478833725845507,Thu Feb 04 23:59:09 +0000 2021,RT @iirman_: Dari Rachel Vennya jd inget kata ...
2,1357478819456720896,Thu Feb 04 23:59:05 +0000 2021,RT @handokotjung: Berhasil membuat banyak neti...
3,1357478818701733888,Thu Feb 04 23:59:05 +0000 2021,RT @yeontinnie: dari rachel vennya kita belaja...
4,1357478707884032001,Thu Feb 04 23:58:39 +0000 2021,RT @mncveronica: Cerita Rachel Vennya dan Okin...
...,...,...,...
1995,1357367016533880832,Thu Feb 04 16:34:50 +0000 2021,VCS brbayar😘\nNew akun 💦\nWa:083191018873 \n#v...
1996,1357366907872047104,Thu Feb 04 16:34:24 +0000 2021,"gapapa wajar lagi musim, apalagi musim ujan gi..."
1997,1357366617651306496,Thu Feb 04 16:33:14 +0000 2021,"Selingkuh memang salah sih, tapi mungkin itu b..."
1998,1357366613327052800,Thu Feb 04 16:33:13 +0000 2021,RT @SpaPanggilan: #Selingkuh #sange cek video ...


In [ ]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweetid      2000 non-null   int64 
 1   createddate  2000 non-null   object
 2   tweet        2000 non-null   object
dtypes: int64(1), object(2)
memory usage: 47.0+ KB


In [ ]:
dT['createddate'] = pd.to_datetime(dT['createddate'])
dT['createddate'] = dT['createddate'].dt.strftime('%Y-%m-%d')
dT['createddate'] = pd.to_datetime(dT['createddate'])

In [2]:
def prepTweet(list_of_tweet):
  # no stopword, will use stopword next step
  clean = []
  for t in list_of_tweet:
    low = t.lower()
    junk = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", low)
    junk_num = re.sub(r'^\d+\s|\d+$|\d+\s|\s\d+', '', junk)
    make = str.maketrans('','',string.punctuation)
    punct = junk_num.translate(make).strip()
    remove_rt = re.sub(r'\Art\s', '', punct)
    clean.append(remove_rt)
  return clean

# because vcs is usually stand for Video Call Sex which is not related
def removeWord(data, word):
  df_filtered = data[~data['tweet'].str.contains(str(word))]
  return df_filtered

In [ ]:
dT['tweet'] = prepTweet(dT['tweet'].values)

In [ ]:
dT = removeWord(dT, 'vcs')
dT = removeWord(dT, 'meatlovers')

In [ ]:
dT

,tweetid,createddate,tweet
0,1357479018472239104,2021-02-04,dari orang yang belajar dari rachel vennya a...
1,1357478833725845507,2021-02-04,dari rachel vennya jd inget kata raditya dika...
2,1357478819456720896,2021-02-04,berhasil membuat banyak netizen belajar rache...
3,1357478818701733888,2021-02-04,dari rachel vennya kita belajar as long as ur...
4,1357478707884032001,2021-02-04,cerita rachel vennya dan okin membawaku untuk...
...,...,...,...
1994,1357367016961802240,2021-02-04,khaaa selingkuh gimana kan aku jomblo
1996,1357366907872047104,2021-02-04,gapapa wajar lagi musim apalagi musim ujan gin...
1997,1357366617651306496,2021-02-04,selingkuh memang salah sih tapi mungkin itu bi...
1998,1357366613327052800,2021-02-04,selingkuh sange cek video surabaya medan ja...


In [ ]:
dT.to_csv('tweet.csv', index=False)

## preprocess tweet part 2

In [5]:
!pip install PySastrawi

     |████████████████████████████████| 215kB 5.1MB/s 


In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
tweet = pd.read_csv('tweet.csv')

In [8]:
tweet

,tweetid,createddate,tweet
0,1357479018472239104,2021-02-04,dari orang yang belajar dari rachel vennya a...
1,1357478833725845507,2021-02-04,dari rachel vennya jd inget kata raditya dika...
2,1357478819456720896,2021-02-04,berhasil membuat banyak netizen belajar rache...
3,1357478818701733888,2021-02-04,dari rachel vennya kita belajar as long as ur...
4,1357478707884032001,2021-02-04,cerita rachel vennya dan okin membawaku untuk...
...,...,...,...
1537,1357367016961802240,2021-02-04,khaaa selingkuh gimana kan aku jomblo
1538,1357366907872047104,2021-02-04,gapapa wajar lagi musim apalagi musim ujan gin...
1539,1357366617651306496,2021-02-04,selingkuh memang salah sih tapi mungkin itu bi...
1540,1357366613327052800,2021-02-04,selingkuh sange cek video surabaya medan ja...


In [3]:
def removeDup(data, column):
  df = data.drop_duplicates(subset=[str(column)])
  return df

def prepTweet2(data, column_to_prep, words_removal):
  df = data
  for word in words_removal:
    df = removeWord(data, word)
  df = removeDup(df, column_to_prep)
  list_of_tweet = df[str(column_to_prep)]
  clean = []
  for val in list_of_tweet:
    token = word_tokenize(val)
    not_stopword = [word for word in token if word not in stop_words]
    clean.append(' '.join(not_stopword))
  df['clean'] = clean
  return df

In [9]:
tweet = prepTweet2(tweet, 'tweet', ['sange'])

In [13]:
tweet.drop('tweet', axis=1, inplace=True)

In [17]:
tweet.rename({'clean':'tweet'}, axis=1, inplace=True)

In [18]:
tweet

,tweetid,createddate,tweet
0,1357479018472239104,2021-02-04,orang belajar rachel vennya belajar orang bela...
1,1357478833725845507,2021-02-04,rachel vennya jd inget raditya dikamenikahlah ...
2,1357478819456720896,2021-02-04,berhasil netizen belajar rachel vennya bikin les
3,1357478818701733888,2021-02-04,rachel vennya belajar as long as ure in relati...
4,1357478707884032001,2021-02-04,cerita rachel vennya okin membawaku mengeluark...
...,...,...,...
1535,1357367144271417346,2021-02-04,dituduh selingkuh malamjumat
1536,1357367049484263425,2021-02-04,selingkuh enak rt setuju
1537,1357367016961802240,2021-02-04,khaaa selingkuh gimana jomblo
1538,1357366907872047104,2021-02-04,gapapa wajar musim musim ujan gini selingkuh


In [19]:
tweet.to_csv('tweet_clean.csv', index=False)

## preprocess user

In [ ]:
dUs = df_user
dUs

,userid,name,screenname,location,accountcreated,follower,friend,verified
0,892670807121330177,Ferdy Agusman,gusmanerdy,Medan,Wed Aug 02 08:58:00 +0000 2017,6,14,0
1,277434635,Karina angelika,karigel13,between his lungs,Tue Apr 05 11:03:15 +0000 2011,417,378,0
2,544170238,Kanjeng Sultan,ayiekkresna,"Kota Surabaya, Jawa Timur",Tue Apr 03 10:42:42 +0000 2012,590,333,0
3,1356180001888976899,UrExtacy,6hndrd6ty6,Bikini Bottom,Mon Feb 01 09:58:22 +0000 2021,2,114,0
4,1168873359741874176,bela,twingkerbelll,NaN,Tue Sep 03 13:08:38 +0000 2019,42,48,0
...,...,...,...,...,...,...,...,...
1995,1356663513024880641,Dania Putri,DaniaPu19568142,NaN,Tue Feb 02 17:59:41 +0000 2021,5,0,0
1996,1270945722259529728,AL,rohfakboi,NaN,Thu Jun 11 05:07:38 +0000 2020,34,105,0
1997,1291683283516850178,Rexsy Aprino Sanggen,rexsy_aprino,"Central Borneo, Indonesia",Fri Aug 07 10:31:18 +0000 2020,5,5,0
1998,1011527885876654080,pijat panggilan,SpaPanggilan,"surabaya, malang, palembang,",Tue Jun 26 08:33:36 +0000 2018,2072,222,0


In [ ]:
dUs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   userid          2000 non-null   int64         
 1   name            2000 non-null   object        
 2   screenname      2000 non-null   object        
 3   location        2000 non-null   object        
 4   accountcreated  2000 non-null   datetime64[ns]
 5   follower        2000 non-null   int64         
 6   friend          2000 non-null   int64         
 7   verified        2000 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 125.1+ KB


In [ ]:
dUs['name'] = dUs['name'].fillna('hai')

In [ ]:
dUs['location'] = dUs['location'].fillna('Indonesia')

In [ ]:
dUs['accountcreated'] = pd.to_datetime(dUs['accountcreated'])
dUs['accountcreated'] = dUs['accountcreated'].dt.strftime('%Y-%m-%d')
dUs['accountcreated'] = pd.to_datetime(dUs['accountcreated'])

In [ ]:
dUs

,userid,name,screenname,location,accountcreated,follower,friend,verified
0,892670807121330177,Ferdy Agusman,gusmanerdy,Medan,2017-08-02,6,14,0
1,277434635,Karina angelika,karigel13,between his lungs,2011-04-05,417,378,0
2,544170238,Kanjeng Sultan,ayiekkresna,"Kota Surabaya, Jawa Timur",2012-04-03,590,333,0
3,1356180001888976899,UrExtacy,6hndrd6ty6,Bikini Bottom,2021-02-01,2,114,0
4,1168873359741874176,bela,twingkerbelll,Indonesia,2019-09-03,42,48,0
...,...,...,...,...,...,...,...,...
1995,1356663513024880641,Dania Putri,DaniaPu19568142,Indonesia,2021-02-02,5,0,0
1996,1270945722259529728,AL,rohfakboi,Indonesia,2020-06-11,34,105,0
1997,1291683283516850178,Rexsy Aprino Sanggen,rexsy_aprino,"Central Borneo, Indonesia",2020-08-07,5,5,0
1998,1011527885876654080,pijat panggilan,SpaPanggilan,"surabaya, malang, palembang,",2018-06-26,2072,222,0


In [ ]:
dUs.to_csv('user.csv', index=False)